In [1]:
# импортируем pandas
import pandas as pd

# импортируем класс TSDataset
from etna.datasets.tsdataset import TSDataset
# импортируем классы для выполнения преобразований
from etna.transforms import (
    StandardScalerTransform,
    MeanTransform, 
    LagTransform,
    DateFlagsTransform)
# импортируем класс Pipeline для 
# выполнения цепочки преобразований
from etna.pipeline import Pipeline
# импортируем класс SMAPE для оценки качества прогнозов
from etna.metrics import SMAPE

# импортируем функцию train_and_evaluate_model() для
# быстрой оценки качества модели и собственный класс
# LGBMMultiSegmentModel
from etna_utils import (train_and_evaluate_model,
                        LGBMMultiSegmentModel)

# отключаем предупреждения
import warnings
warnings.filterwarnings('ignore')

/Users/artemgruzdev/anaconda3/lib/python3.10/site-packages/etna/settings.py:61: UserWarning: `tsfresh` is not available, to install it, run `pip install tsfresh==0.19.0 && pip install protobuf==3.20.1`
  warnings.warn(
<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
# загружаем исторический набор
df = pd.read_csv('Data/demand/train.csv', 
                 parse_dates=['date'])
# переименовываем date в timestamp, sales в target
df.rename(columns={'date': 'timestamp', 
                   'sales': 'target'}, inplace=True)
df

,timestamp,store,item,target
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
...,...,...,...,...
912995,2017-12-27,10,50,63
912996,2017-12-28,10,50,59
912997,2017-12-29,10,50,74
912998,2017-12-30,10,50,62


In [3]:
# загружаем набор новых данных
df_new = pd.read_csv('Data/demand/test.csv', 
                     parse_dates=['date'])
# переименовываем date в timestamp
df_new.rename(columns={'date': 'timestamp'}, inplace=True)
ident = df_new['id']
df_new.drop('id', inplace=True, axis=1)
df_new

,timestamp,store,item
0,2018-01-01,1,1
1,2018-01-02,1,1
2,2018-01-03,1,1
3,2018-01-04,1,1
4,2018-01-05,1,1
...,...,...,...
44995,2018-03-27,10,50
44996,2018-03-28,10,50
44997,2018-03-29,10,50
44998,2018-03-30,10,50


In [4]:
# создаем сегменты - комбинации продуктовой 
# группы и номера магазина
df['segment'] = (df['store'].astype(str) + ' + ' 
                 + df['item'].astype(str))
df_new['segment'] = (df_new['store'].astype(str) + ' + ' 
                     + df_new['item'].astype(str))
df.head()

,timestamp,store,item,target,segment
0,2013-01-01,1,1,13,1 + 1
1,2013-01-02,1,1,11,1 + 1
2,2013-01-03,1,1,14,1 + 1
3,2013-01-04,1,1,13,1 + 1
4,2013-01-05,1,1,10,1 + 1


In [5]:
# присваиваем тип category столбцам store и item
for col in ['store', 'item']:
    df[col] = df[col].astype('category')
    df_new[col] = df_new[col].astype('category')

In [6]:
# формируем набор экзогенных переменных store и item
# для исторического периода
regressor_df = df[['timestamp', 'segment', 'store', 'item']].copy()
regressor_df

,timestamp,segment,store,item
0,2013-01-01,1 + 1,1,1
1,2013-01-02,1 + 1,1,1
2,2013-01-03,1 + 1,1,1
3,2013-01-04,1 + 1,1,1
4,2013-01-05,1 + 1,1,1
...,...,...,...,...
912995,2017-12-27,10 + 50,10,50
912996,2017-12-28,10 + 50,10,50
912997,2017-12-29,10 + 50,10,50
912998,2017-12-30,10 + 50,10,50


In [7]:
# формируем набор экзогенных переменных store и item
# для прогнозируемого периода
regressor_df_new = df_new.copy()
regressor_df_new

,timestamp,store,item,segment
0,2018-01-01,1,1,1 + 1
1,2018-01-02,1,1,1 + 1
2,2018-01-03,1,1,1 + 1
3,2018-01-04,1,1,1 + 1
4,2018-01-05,1,1,1 + 1
...,...,...,...,...
44995,2018-03-27,10,50,10 + 50
44996,2018-03-28,10,50,10 + 50
44997,2018-03-29,10,50,10 + 50
44998,2018-03-30,10,50,10 + 50


In [8]:
# сортируем признаки для последующей конкатенации
regressor_df = regressor_df.sort_index(
    axis=1, ascending=False)
regressor_df_new = regressor_df_new.sort_index(
    axis=1, ascending=False)

In [9]:
# конкатенируем набор с экзогенными переменными 
# для исторического периода и набор с экзогенными 
# переменными для прогнозируемого периода
regressor_df = pd.concat([regressor_df, regressor_df_new], axis=0)
# создаем новые экзогенные переменные
regressor_df['quarter'] = regressor_df['timestamp'].dt.quarter
regressor_df['quarter_start'] = regressor_df['timestamp'].dt.is_quarter_start
regressor_df['quarter_end'] = regressor_df['timestamp'].dt.is_quarter_end
regressor_df

,timestamp,store,segment,item,quarter,quarter_start,quarter_end
0,2013-01-01,1,1 + 1,1,1,True,False
1,2013-01-02,1,1 + 1,1,1,False,False
2,2013-01-03,1,1 + 1,1,1,False,False
3,2013-01-04,1,1 + 1,1,1,False,False
4,2013-01-05,1,1 + 1,1,1,False,False
...,...,...,...,...,...,...,...
44995,2018-03-27,10,10 + 50,50,1,False,False
44996,2018-03-28,10,10 + 50,50,1,False,False
44997,2018-03-29,10,10 + 50,50,1,False,False
44998,2018-03-30,10,10 + 50,50,1,False,False


In [10]:
# подготавливаем исторический набор эндогенных переменных
df.drop(['store', 'item'], axis=1, inplace=True)
df

,timestamp,target,segment
0,2013-01-01,13,1 + 1
1,2013-01-02,11,1 + 1
2,2013-01-03,14,1 + 1
3,2013-01-04,13,1 + 1
4,2013-01-05,10,1 + 1
...,...,...,...
912995,2017-12-27,63,10 + 50
912996,2017-12-28,59,10 + 50
912997,2017-12-29,74,10 + 50
912998,2017-12-30,62,10 + 50


In [11]:
# переводим исторический набор эндогенных 
# переменных в формат TSDataset
df = TSDataset.to_dataset(df)
df

segment,1 + 1,1 + 10,1 + 11,1 + 12,1 + 13,1 + 14,1 + 15,1 + 16,1 + 17,1 + 18,...,9 + 46,9 + 47,9 + 48,9 + 49,9 + 5,9 + 50,9 + 6,9 + 7,9 + 8,9 + 9
feature,target,target,target,target,target,target,target,target,target,target,...,target,target,target,target,target,target,target,target,target,target
timestamp,,,,,,,,,,,,,,,,,,,,,
2013-01-01,13,37,37,33,37,22,42,14,13,38,...,34,6,28,11,9,36,29,30,45,27
2013-01-02,11,34,43,35,31,35,33,11,18,51,...,28,14,38,16,11,44,33,24,43,36
2013-01-03,14,32,34,41,50,26,45,12,15,42,...,41,18,24,20,8,29,19,35,34,25
2013-01-04,13,45,52,45,45,32,39,15,19,50,...,41,15,30,19,15,43,33,35,41,31
2013-01-05,10,35,45,46,49,31,47,22,16,56,...,42,13,33,16,13,53,36,28,49,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-27,14,55,43,47,49,42,62,16,29,54,...,49,19,41,26,18,52,39,44,52,44
2017-12-28,19,63,64,49,68,51,82,24,13,69,...,42,23,36,37,18,73,56,54,76,48


In [12]:
# переводим получившийся набор с экзогенными 
# переменными в формат TSDataset
regressor_df = TSDataset.to_dataset(regressor_df)
regressor_df

segment    1 + 1                                         1 + 10          \
feature     item quarter quarter_end quarter_start store   item quarter   
timestamp                                                                 
2013-01-01     1       1       False          True     1     10       1   
2013-01-02     1       1       False         False     1     10       1   
2013-01-03     1       1       False         False     1     10       1   
2013-01-04     1       1       False         False     1     10       1   
2013-01-05     1       1       False         False     1     10       1   
...          ...     ...         ...           ...   ...    ...     ...   
2018-03-27     1       1       False         False     1     10       1   
2018-03-28     1       1       False         False     1     10       1   
2018-03-29     1       1       False         False     1     10       1   
2018-03-30     1       1       False         False     1     10       1   
2018-03-31     1       1        True         False     1     10       1   

segment                                     ... 9 + 8                      \
feature    quarter_end quarter_start store  ...  item quarter quarter_end   
timestamp                                   ...                             
2013-01-01       False          True     1  ...     8       1       False   
2013-01-02       False         False     1  ...     8       1       False   
2013-01-03       False         False     1  ...     8       1       False   
2013-01-04       False         False     1  ...     8       1       False   
2013-01-05       False         False     1  ...     8       1       False   
...                ...           ...   ...  ...   ...     ...         ...   
2018-03-27       False         False     1  ...     8       1       False   
2018-03-28       False         False     1  ...     8       1       False   
2018-03-29       False         False     1  ...     8       1       False   
2018-03-30       False         False     1  ...     8       1       False   
2018-03-31        True         False     1  ...     8       1        True   

segment                        9 + 9                                          
feature    quarter_start store  item quarter quarter_end quarter_start store  
timestamp                                                                     
2013-01-01          True     9     9       1       False          True     9  
2013-01-02         False     9     9       1       False         False     9  
2013-01-03         False     9     9       1       False         False     9  
2013-01-04         False     9     9       1       False         False     9  
2013-01-05         False     9     9       1       False         False     9  
...                  ...   ...   ...     ...         ...           ...   ...  
2018-03-27         False     9     9       1       False         False     9  
2018-03-28         False     9     9       1       False         False     9  
2018-03-29         False     9     9       1       False         False     9  
2018-03-30         False     9     9       1       False         False     9  
2018-03-31         False     9     9       1        True         False     9  

[1916 rows x 2500 columns]

In [13]:
# создаем объединенный набор
ts = TSDataset(df=df, freq='D', df_exog=regressor_df, 
               known_future='all')
ts

segment    1 + 1                                                1 + 10  \
feature     item quarter quarter_end quarter_start store target   item   
timestamp                                                                
2013-01-01     1       1       False          True     1   13.0     10   
2013-01-02     1       1       False         False     1   11.0     10   
2013-01-03     1       1       False         False     1   14.0     10   
2013-01-04     1       1       False         False     1   13.0     10   
2013-01-05     1       1       False         False     1   10.0     10   
...          ...     ...         ...           ...   ...    ...    ...   
2017-12-27     1       4       False         False     1   14.0     10   
2017-12-28     1       4       False         False     1   19.0     10   
2017-12-29     1       4       False         False     1   15.0     10   
2017-12-30     1       4       False         False     1   27.0     10   
2017-12-31     1       4        True         False     1   23.0     10   

segment                                       ...       9 + 8                \
feature    quarter quarter_end quarter_start  ... quarter_end quarter_start   
timestamp                                     ...                             
2013-01-01       1       False          True  ...       False          True   
2013-01-02       1       False         False  ...       False         False   
2013-01-03       1       False         False  ...       False         False   
2013-01-04       1       False         False  ...       False         False   
2013-01-05       1       False         False  ...       False         False   
...            ...         ...           ...  ...         ...           ...   
2017-12-27       4       False         False  ...       False         False   
2017-12-28       4       False         False  ...       False         False   
2017-12-29       4       False         False  ...       False         False   
2017-12-30       4       False         False  ...       False         False   
2017-12-31       4        True         False  ...        True         False   

segment                 9 + 9                                                 
feature    store target  item quarter quarter_end quarter_start store target  
timestamp                                                                     
2013-01-01     9   45.0     9       1       False          True     9   27.0  
2013-01-02     9   43.0     9       1       False         False     9   36.0  
2013-01-03     9   34.0     9       1       False         False     9   25.0  
2013-01-04     9   41.0     9       1       False         False     9   31.0  
2013-01-05     9   49.0     9       1       False         False     9   30.0  
...          ...    ...   ...     ...         ...           ...   ...    ...  
2017-12-27     9   52.0     9       4       False         False     9   44.0  
2017-12-28     9   76.0     9       4       False         False     9   48.0  
2017-12-29     9   73.0     9       4       False         False     9   54.0  
2017-12-30     9   74.0     9       4       False         False     9   59.0  
2017-12-31     9   92.0     9       4        True         False     9   50.0  

[1826 rows x 3000 columns]

In [14]:
# создаем экземпляр класса StandardScalerTransform
scaler = StandardScalerTransform(in_column='target')

# создаем экземпляр класса LagTransform для генерации лагов
lags = LagTransform(in_column='target', 
                    lags=[90, 120, 150, 180, 210, 240, 
                          270, 300, 330, 360], 
                    out_column='lag')

# создаем экземпляры класса MeanTransform для 
# вычисления среднего по заданному окну
mean90 = MeanTransform(in_column='target', window=90, 
                        out_column='mean90')
mean180 = MeanTransform(in_column='target', window=180, 
                        out_column='mean180')
mean210 = MeanTransform(in_column='target', window=210, 
                        out_column='mean210')
mean240 = MeanTransform(in_column='target', window=240, 
                        out_column='mean240')
mean270 = MeanTransform(in_column='target', window=270, 
                        out_column='mean270')
mean360 = MeanTransform(in_column='target', window=360, 
                        out_column='mean360')

# создаем экземпляр класса DateFlagsTransform 
# для генерации признаков на основе дат
d_flags = DateFlagsTransform(day_number_in_year=True,
                             day_number_in_week=True,
                             day_number_in_month=True,
                             week_number_in_month=True,
                             week_number_in_year=True,
                             month_number_in_year=True,
                             season_number=True,
                             is_weekend=True,
                             out_column='datetime')

In [15]:
# задаем список преобразований/признаков
preprocess = [scaler, lags, mean90, mean180, mean210, mean240, 
              mean270, mean360, d_flags]

# задаем горизонт прогнозирования
HORIZON = 90

# создаем экземпляр класса SMAPE
smape = SMAPE()

# создаем модель LGBMMultiSegmentModel
lgbm_model = LGBMMultiSegmentModel(n_estimators=150, 
                                   learning_rate=0.1,
                                   num_leaves=10,
                                   min_data_in_leaf=120,
                                   subsample=0.8)
# обучаем модель и оцениваем ее качество
train_and_evaluate_model(
    ts=ts,
    model=lgbm_model,
    transforms=preprocess,
    horizon=HORIZON,
    metrics=smape,
    print_plots=False,
    print_metrics=True,
    n_train_samples=None)

[LightGBM] [Warning] min_data_in_leaf is set=120, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=120
3 + 16     12.799516
1 + 10     10.544702
8 + 38      8.566917
10 + 38     8.873962
3 + 26     11.089034
7 + 5      23.166617
2 + 9      10.018343
8 + 23     16.082412
8 + 16     14.104883
1 + 35      9.471480
10 + 23    13.004416
3 + 13      8.611766
1 + 40     15.403773
4 + 29      9.011282
5 + 13     12.229128
8 + 22      7.932302
9 + 48     10.331350
3 + 28      8.417337
9 + 16     15.098450
2 + 14      8.744621
7 + 47     19.314441
1 + 42     14.521090
10 + 12    10.005328
2 + 50      8.917700
7 + 34     18.024114
10 + 30    11.212032
10 + 1     12.855708
9 + 42     14.167873
1 + 29     10.019705
9 + 40     15.561494
6 + 41     19.273485
9 + 13      8.103392
4 + 35      8.942033
5 + 25     10.475857
2 + 19     10.395668
5 + 18      9.111384
10 + 21    11.774174
1 + 1      20.240698
4 + 21     14.184020
3 + 47     15.050271
6 + 30     13.200850
6 + 13     11.1

In [16]:
# создаем конвейер
pipe = Pipeline(
    model=lgbm_model,
    transforms=preprocess,
    horizon=HORIZON)
        
# находим метрики моделей по сегментам 
# по итогам перекрестной проверки
metrics_df, _, _ = pipe.backtest(
    mode='expand', 
    n_folds=4,
    ts=ts, 
    metrics=[smape], 
    aggregate_metrics=True,
    joblib_params=dict(backend='loky'))

[LightGBM] [Warning] min_data_in_leaf is set=120, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=120


In [17]:
# смотрим метрики по первым 8 сегментам
metrics_df.head(8)

,segment,SMAPE
0,1 + 1,18.315264
1,1 + 10,10.273194
2,1 + 11,10.049248
3,1 + 12,10.030497
4,1 + 13,9.160326
5,1 + 14,10.528913
6,1 + 15,9.019105
7,1 + 16,17.475485


In [18]:
# смотрим значение SMAPE, усредненное по сегментам
metric = metrics_df['SMAPE'].mean()
print(f'mean SMAPE: {metric:.4f}')

mean SMAPE: 12.1794


In [19]:
# выполняем преобразования всего исторического набора
ts.fit_transform(preprocess)

# обучаем модель на всем историческом наборе
lgbm_model.fit(ts)

# формируем набор, для которого нужно получить прогнозы,
# длина набора определяется горизонтом прогнозирования,
# по сути мы формируем набор новых данных
future_ts = ts.make_future(HORIZON, preprocess)

# получаем прогнозы для новых данных
forecast_ts = lgbm_model.forecast(future_ts)

# выполняем обратные преобразования прогнозов
forecast_ts.inverse_transform(preprocess)

[LightGBM] [Warning] min_data_in_leaf is set=120, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=120


In [20]:
# превращаем в обычный плоский датафрейм
forecast_ts = forecast_ts.to_pandas(flatten=True)
forecast_ts

,timestamp,segment,target,datetime_day_number_in_month,datetime_day_number_in_week,datetime_day_number_in_year,datetime_is_weekend,datetime_month_number_in_year,datetime_season_number,datetime_week_number_in_month,...,mean180,mean210,mean240,mean270,mean360,mean90,quarter,quarter_end,quarter_start,store
0,2018-01-01,1 + 1,12.584808,1,0,1,False,1,1,1,...,0.460990,0.568661,0.600874,0.584531,0.342744,0.054243,1,False,True,1
1,2018-01-02,1 + 1,13.893766,2,1,2,False,1,1,1,...,0.454386,0.569948,0.596523,0.583928,0.346177,0.063243,1,False,False,1
2,2018-01-03,1 + 1,14.081157,3,2,3,False,1,1,1,...,0.446869,0.566229,0.596518,0.584432,0.347135,0.063921,1,False,False,1
3,2018-01-04,1 + 1,15.214370,4,3,4,False,1,1,1,...,0.443482,0.564635,0.591482,0.586613,0.348932,0.066341,1,False,False,1
4,2018-01-05,1 + 1,15.898003,5,4,5,False,1,1,1,...,0.435817,0.570264,0.595876,0.589371,0.353664,0.063580,1,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,2018-03-27,9 + 9,50.130838,27,1,87,False,3,2,5,...,0.131325,0.228783,0.379544,0.562556,0.718444,-0.089464,1,False,False,9
44996,2018-03-28,9 + 9,51.442340,28,2,88,False,3,2,5,...,0.120750,0.222652,0.379913,0.559538,0.718684,0.015022,1,False,False,9
44997,2018-03-29,9 + 9,55.417310,29,3,89,False,3,2,5,...,0.110661,0.219666,0.378985,0.550685,0.723535,0.026021,1,False,False,9
44998,2018-03-30,9 + 9,58.642238,30,4,90,False,3,2,5,...,0.108328,0.209541,0.379793,0.553401,0.724526,-0.270939,1,False,False,9


In [21]:
# формируем посылку
subm = (
    df_new.drop(['store', 'item'], axis=1)
    .merge(forecast_ts, on=['timestamp', 'segment'])
    ['target'].reset_index()
    .rename({'index': 'id', 'target': 'sales'}, axis=1)
    )
subm

,id,sales
0,0,12.584808
1,1,13.893766
2,2,14.081157
3,3,15.214370
4,4,15.898003
...,...,...
44995,44995,69.047680
44996,44996,71.076568
44997,44997,75.676394
44998,44998,80.229211


In [22]:
# смотрим статистики
subm['sales'].describe()

count    45000.000000
mean        46.786958
std         23.189439
min          7.310673
25%         27.650760
50%         43.416620
75%         62.329634
max        134.331404
Name: sales, dtype: float64

In [23]:
# записываем посылку в CSV-файл
subm.to_csv('kaggle_store_item_demand_submission.csv', 
            index=False)

<img src='img/submit.png'>